# Use AWS Components

Using all local component is by far the cheapest route, but some time you want to use things that are stored in AWS.
In order to use the AWS components from this docker image, you'll need to make sure you setup the AWS credential properly.

In this notebook, we will look at:

* Access to AWS Glue Catalog
* Access data in S3 bucket
* Write modified data to S3 bucket

## System Args

Similar to full local setup, we will need to setup the fake parameter.

In [1]:
import sys
sys.argv += ['--JOB_NAME', 'glue_script']

## Import Libs

The first thing we do is to import the necessary library for our application. 
Since we installed Spark and Glue in the docker container, we are able to load the spark and glue modules here.

In [2]:
# Import in all of the pyspark functions in
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType, StructField, StructType, DataType, DecimalType, DoubleType
from pyspark.context import SparkContext

# Import glue module components in
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.utils import getResolvedOptions
from awsglue.dynamicframe import DynamicFrame

##  AWS Credentials

In [3]:
import os
import getpass

aws_profile = getpass.getpass(prompt="AWS Profile: ")
aws_region = input(prompt="AWS Region: ")

os.environ["AWS_PROFILE"] = aws_profile
os.environ["AWS_REGION"] = aws_region

AWS Profile: ········
AWS Region: us-west-2


## Args

Load the configuration args using the glue provided function

In [4]:
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

## Setup the Spark Context

In [5]:
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)
sc.getConf().getAll()

[('spark.executor.extraClassPath', '/opt/aws-glue-libs/jarsv1/*'),
 ('spark.app.id', 'local-1597640676340'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.driver.extraClassPath', '/opt/aws-glue-libs/jarsv1/*'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '7b606bae576f'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.driver.port', '40465')]

## Glue Catalog

You will need to upload the hygdata_v3.csv to a S3 bucket and run glue crawler over the data.

**TODO** some instruction on upload/running crawler.

In [6]:
glue_database = input("Glue Catalog Database: ")
glue_table = input("Glue Catalog Table: ")

etl_test_data = glueContext.create_dynamic_frame.from_catalog(
    database=glue_database, table_name=glue_table)
etl_test_data.toDF().printSchema()

Glue Catalog Database: etl_test
Glue Catalog Table: etl_test_data
root
 |-- id: long (nullable = true)
 |-- gl: string (nullable = true)
 |-- bf: string (nullable = true)
 |-- proper: string (nullable = true)
 |-- ra: double (nullable = true)
 |-- dec: double (nullable = true)
 |-- dist: double (nullable = true)
 |-- pmra: double (nullable = true)
 |-- pmdec: double (nullable = true)
 |-- rv: double (nullable = true)
 |-- mag: double (nullable = true)
 |-- absmag: double (nullable = true)
 |-- spect: string (nullable = true)
 |-- ci: double (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)
 |-- vx: double (nullable = true)
 |-- vy: double (nullable = true)
 |-- vz: double (nullable = true)
 |-- rarad: double (nullable = true)
 |-- decrad: double (nullable = true)
 |-- pmrarad: double (nullable = true)
 |-- pmdecrad: double (nullable = true)
 |-- bayer: string (nullable = true)
 |-- con: string (nullable = true)
 |-- com

We won't dive too deep into what the crawler found, but in general it should be the same as what we specified manually.

## Process Data

We will process the data exactly the same as we did in the local setup.
The output should be the same, since we are working with the same data.

In [7]:
print("Original {}".format(etl_test_data.toDF().count()))
filtered_data = etl_test_data.filter(f = lambda x: x['lum'] >= 0.99 and x['lum'] <= 1.1, transformation_ctx="filtered_data")
print("Filtered {}".format(filtered_data.toDF().count()))

Original 119614
Filtered 705


In [8]:
removed_bad_data = filtered_data.filter(f = lambda x: x['dist'] is not None and x['dist'] < 100000 and x['dist'] >=0, transformation_ctx="removed_bad_data")
print("Filtered bad data: {}".format(removed_bad_data.toDF().count()))

Filtered bad data: 705


In [9]:
def add_light_year(d):
    d['light_year'] = d['dist'] * 3.262
    return d
with_light_year = removed_bad_data.map(f=add_light_year, transformation_ctx="with_light_year")
with_light_year.select_fields(["id", "hr", "bf", "dist", "light_year"]).toDF().orderBy("light_year").show()

+------+---------+-------+------------------+----+
|    id|       bf|   dist|        light_year|  hr|
+------+---------+-------+------------------+----+
|     0|         |    0.0|               0.0|null|
| 15333|  Zet2Ret|12.0322|        39.2490364|1010|
| 80095|         |12.7779|        41.6815098|6094|
| 79431|18    Sco|13.9005|45.343430999999995|6060|
| 42319| 3Pi 1UMa|14.3554|46.827314799999996|3391|
| 74952|  Nu 2Lup|14.8126|        48.3187012|5699|
| 19028|39    Tau|16.9377|        55.2507774|1262|
| 62014|10    CVn|17.3762|        56.6811644|4845|
| 43602|         |17.3853|56.710848600000006|3538|
| 98643|         |17.7274|        57.8267788|7644|
| 98605|         |18.7899|        61.2926538|7683|
| 84783|         |19.5236|63.685983199999995|6465|
|  1799| 9    Cet|20.8638| 68.05771560000001|  88|
|119398|         |20.9205|         68.242671|null|
|101710|         |20.9468|        68.3284616|7914|
|117998|         |21.0482|        68.6592284|null|
| 18366|         |21.9829| 71.7

## Saving

Finally we want to save the file back to s3.
Unlike our local setup, we will use glueContext to save the file back up to s3.

you can easily use spark to save the file backup, but we are using glue here after all.

In [10]:
s3_bucket = getpass.getpass("Bucket: ")

glueContext.write_dynamic_frame.from_options(
    frame=with_light_year, 
    connection_type="s3",
    connection_options={
        "path": "s3://{}/processed/".format(s3_bucket),
    },
    format="parquet",
    transformation_ctx="process_data"
)


Bucket: ········


Now you can crawl over the newly created parquet file and query it from Athena!